# Import necessary libraries

In [2]:
# Import necessary libraries
from datasets import load_dataset
import pandas as pd
from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationalRetrievalChain


# Title for the notebook


In [3]:
# Title for the notebook
print("🩺 Medical Q&A Chatbot using LangChain, Chroma & FLAN-T5")


🩺 Medical Q&A Chatbot using LangChain, Chroma & FLAN-T5


# Load and initialize the system


In [4]:
# Load and initialize the system
def load_and_initialize_system():
    print("Step 1: Loading and preprocessing dataset...")
    data = load_dataset("keivalya/MedQuad-MedicalQnADataset", split='train')
    df = data.to_pandas()
    df = df[:100]  # For demo

    print("Dataset loaded.")

    # Step 2: Document loader
    print("Step 2: Loading documents...")
    df_loader = DataFrameLoader(df, page_content_column="Answer")
    documents = df_loader.load()

    # Step 3: Split documents
    print("Step 3: Splitting text into chunks...")
    text_splitter = CharacterTextSplitter(chunk_size=1250, separator="\n", chunk_overlap=100)
    split_texts = text_splitter.split_documents(documents)

    # Step 4: Generate embeddings
    print("Step 4: Generating embeddings using HuggingFace...")
    embedder = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    # Step 5: Create Chroma DB
    print("Step 5: Creating Chroma vector store...")
    chroma_db = Chroma.from_documents(split_texts, embedder, persist_directory="chromadb")

    # Step 6: Load seq2seq model
    print("Step 6: Loading FLAN-T5 model...")
    hf_pipeline = pipeline(
        "text2text-generation",
        model="google/flan-t5-large",  # Better model for medical Q&A
        max_new_tokens=512,
        temperature=0.7
    )
    llm = HuggingFacePipeline(pipeline=hf_pipeline)

    # Step 7: Setup memory
    memory = ConversationBufferWindowMemory(memory_key='chat_history', k=4, return_messages=True)

    # Step 8: Create Conversational Retrieval Chain
    print("Step 8: Building QA Chain...")
    qa_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=chroma_db.as_retriever(),
        memory=memory
    )

    print("✅ System is ready! You can now ask your medical questions.")
    return qa_chain

# Load the system
qa_chain = load_and_initialize_system()

# Function to ask questions
def ask_question(question):
    formatted_question = (
        f"Act like a professional doctor. Based on the context and your knowledge, "
        f"answer clearly:\n\nQuestion: {question}"
    )
    result = qa_chain.invoke({"question": formatted_question})
    return result["answer"]


Step 1: Loading and preprocessing dataset...
Dataset loaded.
Step 2: Loading documents...
Step 3: Splitting text into chunks...
Step 4: Generating embeddings using HuggingFace...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cpu


Step 8: Building QA Chain...
✅ System is ready! You can now ask your medical questions.


# Example usage

In [ ]:
# Load the system
qa_chain = load_and_initialize_system()

# Function to ask questions
def ask_question(question):
    formatted_question = (
        f"Act like a professional doctor. Based on the context and your knowledge, "
        f"answer clearly:\n\nQuestion: {question}"
    )
    result = qa_chain.invoke({"question": formatted_question})
    return result["answer"]

# Example usage
questions = [
    "I have a patient with persistent cough and weight loss. How can I confirm a diagnosis of tuberculosis?",
    "My patient presents with acute chest pain. What tests should I order to rule out myocardial infarction?",
    "I have a patient with persistent fever and lymphadenopathy. How do I confirm lymphoma?"
]

# Ask a question
for q in questions:
    answer = ask_question(q)
    print(f"Question: {q}\nAnswer: {answer}\n")


Step 1: Loading and preprocessing dataset...
Dataset loaded.
Step 2: Loading documents...
Step 3: Splitting text into chunks...
Step 4: Generating embeddings using HuggingFace...
Step 5: Creating Chroma vector store...
Step 6: Loading FLAN-T5 model...


Device set to use cpu


Step 8: Building QA Chain...
✅ System is ready! You can now ask your medical questions.


Token indices sequence length is longer than the specified maximum sequence length for this model (516 > 512). Running this sequence through the model will result in indexing errors
/home/sagaryadav/anaconda3/envs/chatbot/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Question: I have a patient with persistent cough and weight loss. How can I confirm a diagnosis of tuberculosis?
Answer: Diagnosis often is made by ruling out other infections that cause similar symptoms.



/home/sagaryadav/anaconda3/envs/chatbot/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/sagaryadav/anaconda3/envs/chatbot/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
